<img src="https://global.utsa.edu/tec-partnership/images/logos/logotipo-horizontal-azul-transparente.png"  width="600">

## **Actividad 3 | Aprendizaje supervisado y no supervisado**
### **Análisis de grandes volúmenes de datos (Gpo 10)**
#### Tecnológico de Monterrey
---
*   NOMBRE: Paulina Escalante Campbell
*   MATRÍCULA: A01191962
---
### **Objetivo**

Aplicar algoritmos de aprendizaje supervisado y no supervisado mediante PySpark para la resolución de problemas en análisis de datos, fomentando el desarrollo de habilidades prácticas en el manejo y procesamiento eficiente de grandes conjuntos de datos.

### **1. Introducción**

En el espacio de Big Data, existen tres principales algoritmos de aprendizaje, supervisado, no supervisado y por refuerzo. Estos algoritmos son aplicados a muestras de la población y poder predecir y analizar cantidades enormes de datos. Esta actividad se enfoca en los primeros dos ya que son utilizados generalmente en PySpark.

El aprendizaje **supervisado** es un modelo de aprendizaje automático donde los algoritmos tienen una definición muy simple y estricta de los datos a analizar y los resultados son conocidos y categorizados. Un ejemplo muy básico es el análisis de rayos-x, la entrada es una imagen de rayos-x y la salida es la interpretación de si existe una enfermedad o no. Es un modelo predictor donde ya se conocen los resultados. Algunos algoritmos representativos de este enfoque incluyen:
- Regresión logística
- Árboles de decisión
- Random Forest
- Máquinas de soporte vectorial (SVM)
- Redes neuronales artificiales
  
En PySpark, se pueden implementar varios de estos algoritmos a través del módulo `pyspark.ml.classification` y `pyspark.ml.regression`. Por ejemplo, PySpark ofrece soporte nativo para `LogisticRegression`, `DecisionTreeClassifier`, `RandomForestClassifier`, y `GBTClassifier`, entre otros.

El aprendizaje **no supervisado** es un poco más complejo ya que se usan datos no estructurados y el resultado final no es conocido de antemano.  Se usa normalmente para identificar grupos de datos o categorías. Depende mucho de los datos y las relaciones escondidas que existen entre ellos. Un ejemplo es el uso de estos algoritmos en marketing en e-commerce. Empresas como Amazon y Walmart usan estos algoritmos para predecir lo que un cliente va a comprar a partir de un producto en su carrito, si alguien compra harina, el algoritmo podría predecir basado en la demográfica del cliente que el cliente quiere manzanas o en otros casos cebolla. Algunos de los algoritmos más representativos incluyen:
- K-means
- Clustering jerárquico
- DBSCAN
- Modelos de mezcla gaussiana
- Análisis de componentes principales (PCA)

En el ecosistema de PySpark, estos métodos están disponibles en el módulo `pyspark.ml.clustering` (por ejemplo, KMeans) y `pyspark.ml.feature` (para técnicas como PCA).

#### **Tabla comparativa**

| Característica                | Aprendizaje Supervisado                                 | Aprendizaje No Supervisado                                 |
|----|----------------------------------------------------------|-------------------------------------------------------------|
| **Definición**               | El modelo aprende a partir de datos etiquetados          | El modelo explora datos sin etiquetas para encontrar patrones |
| **Objetivo principal**       | Predecir una etiqueta o valor conocido                   | Descubrir patrones, grupos o asociaciones         |
| **Ejemplos de uso**          | Diagnóstico médico, detección de fraude | Segmentación de clientes, análisis de comportamiento |
| **Algoritmos comunes**       | Regresión logística, árboles de decisión, random forest  | K-means, clustering jerárquico, PCA                         |
| **PySpark**| `LogisticRegression`, `RandomForestClassifier`, etc. | `KMeans`, `PCA`                                 |
| **Tipo de datos necesario**  | Datos con variables de entrada y una etiqueta de salida conocida | Solo variables de entrada, sin etiquetas                     |

Por último, un ejemplo típico de aprendizaje no supervisado es la segmentación de clientes en marketing digital. A través del análisis de patrones de compra y navegación, es posible identificar diferentes grupos de clientes con comportamientos similares, lo que permite personalizar marketing y otras recomendaciones. Por otro lado, un ejemplo típico para aprendizajo supervisado sería un modelo que puede predecir si un correo es spam o no spam.

#### **Supervisado vs. No Supervisado**
![](https://lakshaysuri.wordpress.com/wp-content/uploads/2017/03/sup-vs-unsup.png?w=448)

A continuación, usando el dataset de e-commerce usado en actividades previas podremos explorar lo que es aprendizajo supervisado y no supervisado y cómo prepar los datos para tener un modelo robusto y preciso

#### **Imports**


In [64]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, when, split, col, lit, max, min, expr
from pyspark.sql.functions import to_date, var_samp, variance, var_pop, month, to_timestamp, dayofweek
from pyspark.sql.functions import hour, month
from pyspark.sql.types import NumericType, IntegerType, FloatType
from pyspark.sql.functions import col, round, concat_ws, desc, when, concat
from pyspark.sql import functions as F

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

from datetime import datetime

### **2. Selección de los datos**

Para esta actividad, se propone recolectar una muestra de dimensión contenida (para evitar que los tiempos de procesamiento sean altos) a partir de la base de datos que estás trabajando en tu proyecto. Para ello y tomando como base la actividad previa en la cual has implementado códigos que permiten obtener particiones de la base de datos global D que cumplen con los criterios de las variables de caracterización identificadas, se propone que recuperes un número limitado de instancias de cada partición (aplicando la técnica de muestreo que propusiste en el Módulo 3, Proyecto Base de datos de Big Data, paso 4), lo que te permitirá construir una muestra M a partir de la unión de las instancias que se recuperan de este proceso.

In [2]:
spark = SparkSession.builder \
    .appName("Maestria_evidencia1") \
    .config("spark.driver.memory", "64g") \
    .config("spark.executor.memory", "32g") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.default.parallelism", "16") \
    .config("spark.driver.maxResultSize", "16g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Mejores tablas
#spark, comentando el comando del environment para reducir el ruido del notebook

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/25 20:29:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/25 20:29:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

####
#### La base de datos ha sido limpiada y modificada a este punto
#### 
####
file_path = "/Users/pauescalante/Documents/Maestria/Trimestre 7/BigData/big-data-act/DataModified/expanded_database_ecommerce"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Mostrar cuantos registros se tienen inicialmete para en el futuro reducir a una dimensión contenida
initial_total_count = df.count()
print(f"Número total de registros: {initial_total_count}")

25/05/25 20:29:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
[Stage 3:=================================================>    (134 + 12) / 146]

Número total de registros: 109950731


In [4]:
# Imprimiendo el esquema del Dataframe
df.printSchema()

root
 |-- event_time: date (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- event_time_ts: timestamp (nullable = true)
 |-- parent_category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- price_bucket: string (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- is_weekend: boolean (nullable = true)



In [5]:
# Mostrar el primer registros de ejemplo para visualizar las columnas
df.show(n=1,truncate=False, vertical=True)

-RECORD 0-----------------------------------------------
 event_time      | 2019-11-17                           
 event_type      | view                                 
 product_id      | 5300440                              
 category_id     | 2053013563173241677                  
 brand           | vitek                                
 price           | 17.76                                
 user_id         | 513341639                            
 user_session    | d9544029-2739-4d16-9cac-79650460d9f0 
 event_time_ts   | 2019-11-17 05:35:32                  
 parent_category | None                                 
 subcategory     | None                                 
 price_bucket    | low                                  
 day_of_week     | 1                                    
 is_weekend      | true                                 
only showing top 1 row



### **3. Preparación de los datos**

En esta etapa, se deberán de aplicar estrategias de corrección sobre los datos que integran a la muestra M que se ha preparado en el paso previo, de tal forma que de deje un conjunto M listo para ser procesado por los algoritmos de aprendizaje a aplicar. Para ello se deben de considerar pasos como: corrección de registros / columnas con valores nulos, identificación de valores atípicos, transformación de los tipos de datos, etc. Con lo anterior, se tendrá una muestra M pre-procesada.

Etapa donde se pre-procesa la muestra M, para corregir formatos e inconsistencias de cualquier índole que tenga la muestra original. Se deberá de documentar los pasos que se implementen para resolver esta etapa.

Pre-procesa la muestra M corrigiendo todos los formatos e inconsistencias con documentación detallada y justificada de cada paso.

In [6]:
# En este punto en actividades previas se limpiaron un poco los datos, para validar que no hay nulls
print("\n--- Análisis de valores nulos ---")
for column in df.columns:
    count = df.filter(col(column).isNull()).count()
    if count > 0:
        print(f"Columna '{column}' tiene {count} valores null")
    else:
        print(f"Columna '{column}' no tiene valores null")


--- Análisis de valores nulos ---


Columna 'event_time' no tiene valores null


Columna 'event_type' no tiene valores null


Columna 'product_id' no tiene valores null


Columna 'category_id' no tiene valores null


...

Columna 'subcategory' no tiene valores null


Columna 'price_bucket' no tiene valores null


Columna 'day_of_week' no tiene valores null


[Stage 46:=====================================================>(144 + 2) / 146]

Columna 'is_weekend' no tiene valores null


In [7]:
# Análisis de valores atípicos en precio, para justificar usar price_bucket en vez de precio
print("\n--- Análisis de valores atípicos (precio) ---")
p_stats = df.select("price").describe()
p_stats.show()


--- Análisis de valores atípicos (precio) ---


[Stage 49:=====================================================>(144 + 2) / 146]

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|        109950731|
|   mean|291.6348043534796|
| stddev|356.6799794159954|
|    min|              0.0|
|    max|          2574.07|
+-------+-----------------+



In [8]:
# Ya que tenemos una varianza alta en precios, podemos observar los cuartiles para entender un poco mas los outliers del precio
# Calcular outliers usando IQR
quantiles = df.select("price").approxQuantile("price", [0.25, 0.75], 0.05)
Q1, Q3 = quantiles[0], quantiles[1]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_count = df.filter((col("price") < lower_bound) | (col("price") > upper_bound)).count()
outlier_pct = (outliers_count / df.count()) * 100

# Esta alta dispersión justifica técnicamente el uso de price_bucket en lugar del precio absoluto,
# ya que las categorías Low/Medium/High capturan mejor los segmentos de mercado naturales y mejorarán
# el rendimiento de los modelos de machine learning
print("\n--- Análisis de outliers en precio: ---")
print(f"Rango normal: ${lower_bound:.2f} - ${upper_bound:.2f}")
print(f"Outliers: {outliers_count:,} ({outlier_pct:.2f}%)")

[Stage 57:=================================================>   (136 + 10) / 146]


--- Análisis de outliers en precio: ---
Rango normal: $-336.79 - $734.97
Outliers: 11,488,995 (10.45%)


In [9]:
# Distribución de price_bucket en porcentajes antes del sampling para verificar la muestra
price_bucket_distribution = (
    df.groupBy("price_bucket")
    .count()
    .withColumn("total", lit(initial_total_count))
    .withColumn("percentage", round((col("count") / col("total")) * 100, 2))
    .select("price_bucket", "count", "percentage")
    .orderBy("price_bucket")
)

print("\n--- Distribución completa price_bucket: ---")
price_bucket_distribution.show()


--- Distribución completa price_bucket: ---


[Stage 60:=====================================================>(144 + 2) / 146]

+------------+--------+----------+
|price_bucket|   count|percentage|
+------------+--------+----------+
|        high|32721216|     29.76|
|         low|37649697|     34.24|
|      medium|39579818|      36.0|
+------------+--------+----------+



In [10]:
# Distribución de event_type en porcentajes antes del sampling
event_type_distribution = (
    df.groupBy("event_type")
    .count()
    .withColumn("total", lit(initial_total_count))
    .withColumn("percentage", round((col("count") / col("total")) * 100, 2))
    .select("event_type", "count", "percentage")
    .orderBy("event_type")
)

# La distribución de estos valores también es esparada ya que no muchos clientes compran pero muchos visitan la página web
print("\n--- Distribución completa event_type: ---")
event_type_distribution.show()


--- Distribución completa event_type: ---


[Stage 63:=====================================================>(144 + 2) / 146]

+----------+---------+----------+
|event_type|    count|percentage|
+----------+---------+----------+
|      cart|  3955434|       3.6|
|  purchase|  1659788|      1.51|
|      view|104335509|     94.89|
+----------+---------+----------+



In [11]:
# Calcular muestras M pre-procesada, en la actividad anterior definimos que SRS (Simple Random Sampling)
# Ya que tenemos una poblacion muy grande (approx. 109950731, podemos usar un 0.01% de muestra y tener un número significativo de datos)
print("\n--- SRS sampling: ---")
sample_df = df.sample(fraction=0.0001)
total_count_sample = sample_df.count()
print(f"New sample size: {total_count_sample})")

#Variables de Caracterización Seleccionadas
#event_type: Representa el funnel de conversión (view → cart → purchase)
#price_bucket: Segmenta productos por rango de precio (low/medium/high)


--- SRS sampling: ---


[Stage 66:================================================>    (133 + 12) / 146]

New sample size: 10861)


In [12]:
# Calcular distribuciónes nuevamente del sample
sample_price_bucket_distribution = (
    sample_df.groupBy("price_bucket")
    .count()
    .withColumn("total", lit(total_count_sample))
    .withColumn("percentage", round((col("count") / col("total")) * 100, 2))
    .select("price_bucket", "count", "percentage")
    .orderBy("price_bucket")
)

sample_event_type_distribution = (
    sample_df.groupBy("event_type")
    .count()
    .withColumn("total", lit(total_count_sample))
    .withColumn("percentage", round((col("count") / col("total")) * 100, 2))
    .select("event_type", "count", "percentage")
    .orderBy("event_type")
)

# Se puede observar que el sample tiene distribuciónes similares a la poblacion
# Los nuevos valores son sample_df y total_count_sample de ahora en adelante
print("\n--- Distribución del sample price_bucket: ---")
sample_price_bucket_distribution.show()

print("\n--- Distribución del sample event_type: ---")
sample_event_type_distribution.show()


--- Distribución del sample price_bucket: ---


+------------+-----+----------+
|price_bucket|count|percentage|
+------------+-----+----------+
|        high| 3225|     29.69|
|         low| 3721|     34.26|
|      medium| 3915|     36.05|
+------------+-----+----------+


--- Distribución del sample event_type: ---


[Stage 72:=====================================================>(144 + 2) / 146]

+----------+-----+----------+
|event_type|count|percentage|
+----------+-----+----------+
|      cart|  398|      3.66|
|  purchase|  154|      1.42|
|      view|10309|     94.92|
+----------+-----+----------+



In [13]:
# Valores distintos de columnas categorícas para ver si se puede normalizar o agrupar mas información 
categoricas_columnas= ['subcategory', 'parent_category', 'brand']
output = ""
for column in categoricas_columnas:
    # Cuantos valores existen
    distinct_count = sample_df.select(column).distinct().count()
    
    output += f"\nColumna: '{column}' — {distinct_count} valores distintos\n"
    output += "-" * 50 + "\n"

print(output)

[Stage 87:=====================================================>(144 + 2) / 146]


Columna: 'subcategory' — 114 valores distintos
--------------------------------------------------

Columna: 'parent_category' — 14 valores distintos
--------------------------------------------------

Columna: 'brand' — 960 valores distintos
--------------------------------------------------



In [14]:
# Para brands, si se tienen menos de 50 registros, usamos "Others" asi reducimos variabilidad de brands
min_count = 50
brands_over_min_count = (
    sample_df.groupBy("brand")
    .count()
    .filter(col("count") >= min_count)
    .select("brand")
)

# Usando esta lista hacemos un filtering
brands_list = [row["brand"] for row in brands_over_min_count.collect()]

sample_df = sample_df.withColumn(
    "brand",
    when(col("brand").isin(brands_list), col("brand")).otherwise("others")
)

print("\n--- Reducir variabilidad de brand del sample: ---")
# Cuantos valores existen en el nuevo df, reducimos de 2194 a 294
# Es una manera de normalizar el valor cetagórico de brand
distinct_count_new = sample_df.select("brand").distinct().count()
print(f"\nColumna brand: — {distinct_count_new} valores distintos\n")


--- Reducir variabilidad de brand del sample: ---


[Stage 96:=====================================================>(144 + 2) / 146]


Columna brand: — 32 valores distintos



### **4. Preparación del conjunto de entrenamiento y prueba**

Para esta etapa, la muestra M será divida en un conjunto de entrenamiento y prueba. Para ello, deberás proponer una técnica de muestreo que te permita construir el conjunto de entrenamiento y prueba minimizando el riesgo de inyección de sesgos. Ten en cuenta que, para este punto, deberás de tener en claro el porcentaje de división a utilizar, el cual se deberá de justificar.

In [ ]:
# sample_df tiene la muestra de la población del data set original
# Para separar la muestra entre sets de entrenamiento y prueba se usa un 80:20
# Es el estándar en ML y un buen balance
print("\n-- Entrenamiento/Prueba ---")
train_ratio = 0.8
test_ratio = 0.2
random_seed = 42

# Establece el número de particiones que se usarán al hacer operaciones como shuffle (por ejemplo, en joins, agregaciones o splits).
# Un número mayor puede mejorar la distribución de los datos en clústeres grandes, pero también aumentar el uso de recursos.
spark.conf.set("spark.sql.shuffle.partitions", "200") # puede alterar los resultados, empezamos con un número mayor

# Divide aleatoriamente el DataFrame `sample_df` en dos subconjuntos: uno para entrenamiento y otro para prueba.
train_data,test_data = sample_df.randomSplit([train_ratio,test_ratio], seed = random_seed)

# Imprime cuántas instancias hay en el conjunto de entrenamiento y cuántas en el conjunto de prueba.
print(f"""Existen {train_data.count()} instancias en el conjunto train, y {test_data.count()} en el conjunto test""")


-- Entrenamiento/Prueba ---


[Stage 105:====================================================>(144 + 2) / 146]

Existen 8667 instancias en el conjunto train, y 2194 en el conjunto test


In [ ]:
# Verificar proporciones de los sets de training y de test con event_type, ya que esta columna tiene una distribución esperada
train_dist = train_data.groupBy("event_type").count().withColumnRenamed("count", "train_count")
test_dist = test_data.groupBy("event_type").count().withColumnRenamed("count", "test_count")
verification = train_dist.join(test_dist, "event_type")

# Las distribuciones son esperadas, con view > cart > purchase
print("\n--- Verificación de proporciones event_type: ---")
verification.show()


--- Verificación de proporciones event_type: ---


[Stage 110:===================================================> (143 + 3) / 146]

+----------+-----------+----------+
|event_type|train_count|test_count|
+----------+-----------+----------+
|  purchase|        119|        35|
|      view|       8228|      2081|
|      cart|        320|        78|
+----------+-----------+----------+



In [22]:
# Verificar proporciones de los sets de training y de test con event_type, ya que esta columna tiene una distribución normal
train_dist_2 = train_data.groupBy("price_bucket").count().withColumnRenamed("count", "train_count")
test_dist_2 = test_data.groupBy("price_bucket").count().withColumnRenamed("count", "test_count")
verification_2 = train_dist_2.join(test_dist_2, "price_bucket")

# Las distribuciones son esperadas, con un 33% aproximado en cada categoría 
print("\n--- Verificación de proporciones price_bucket: ---")
verification_2.show()


--- Verificación de proporciones price_bucket: ---


[Stage 122:===================================================> (142 + 4) / 146]

+------------+-----------+----------+
|price_bucket|train_count|test_count|
+------------+-----------+----------+
|         low|       2996|       725|
|        high|       2566|       659|
|      medium|       3105|       810|
+------------+-----------+----------+



### **5. Construcción de modelos de aprendizaje supervisado y no supervisado**

Para este punto realizarás dos experimentos separados, dónde se aplicará un algoritmo de aprendizaje supervisado y uno de aprendizaje no supervisado sobre la muestra M. Para el caso de aprendizaje supervisado, se deberá de identificar cuál es la variable objetivo (columna) de aprendizaje, mientras que, para el caso de aprendizaje no supervisado, se debe de seleccionar todas las columnas que se desean considerar como características bajo las cuales se realizará el proceso de agrupamiento. Usando las implementaciones correspondientes de PySpark, se deberá de ejecutar el aprendizaje correspondiente a partir de la invocación de las funciones respectivas. Para este ejercicio, se deberá seleccionar un criterio básico para medir la calidad del resultado obtenido, dependiendo de cada tipo de aprendizaje implementado. La elección quedará a juicio de cada estudiante.

#### 5.1 Aprendizaje Supervisado

Para el problema de aprendizaje supervisado haremos un análisis binario de predicción: compra o no compra

- Variable objetivo: label (1 = purchase, 0 = no purchase)
- Desafío: Dataset altamente desbalanceado (1.5% positivos)

In [48]:
# Preparar la variable objetivo "label" que identifica si es compra o no
supervised_df = train_data.withColumn("label", 
                                   when(col("event_type") == "purchase", 1.0)
                                   .otherwise(0.0))

print("Variable objetivo: label (1=purchase, 0=no purchase)")

# Seleccionar características, en este caso tenemos variables numéricas y categóricas
feature_cols = ["price", "day_of_week"]
categorical_cols = ["brand", "parent_category", "price_bucket"]

print(f"Características numéricas: {feature_cols}")
print(f"Características categóricas: {categorical_cols}")

# Revisar resultado
sample_df.select("event_time", "day_of_week").show(5)

Variable objetivo: label (1=purchase, 0=no purchase)
Características numéricas: ['price', 'day_of_week']
Características categóricas: ['brand', 'parent_category', 'price_bucket']
+----------+-----------+
|event_time|day_of_week|
+----------+-----------+
|2019-11-17|          1|
|2019-11-17|          1|
|2019-11-17|          1|
|2019-11-17|          1|
|2019-11-17|          1|
+----------+-----------+
only showing top 5 rows



In [49]:
# Preparación del dataframe para ser procesado con algoritmos de ML en PySpark
# Se usa VectorAssembler para generar una o más columnas en la cual, se "encapsulan" en un vector único
# los valores de los descriptores a usar en el proceso de aprendizaje.

# Indexar las variables categóricas (String → Numérico).
# Se crea una lista de transformadores StringIndexer, uno por cada columna categórica.
# `handleInvalid="skip"` evita errores si hay valores nulos o inesperados.
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_indexed", handleInvalid="skip") 
            for col in categorical_cols]

# Ensamblar todas las variables numéricas y categóricas indexadas en una sola columna de características.
# Esto es necesario porque PySpark ML requiere una sola columna de entrada (`features`) de tipo vector.
feature_cols_final = feature_cols + [f"{col}_indexed" for col in categorical_cols]
assembler = VectorAssembler(inputCols=feature_cols_final, outputCol="features")

# Escalar las características para normalizar los valores.
# Esto mejora el rendimiento de muchos algoritmos de ML
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [50]:
# MODELO 1: Regresión Logística
# Baseline estándar para clasificación binaria
print("\n--- Modelo 1: Regresión Logística ---")

# Se especifican las columnas de entrada (`scaledFeatures`) y etiqueta (`label`), y el número máximo de iteraciones.
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="label", maxIter=10)

# Se construye un pipeline que incluye:
# 1. Indexado de columnas categóricas
# 2. Ensamblado de características
# 3. Escalado de variables
# 4. El modelo de regresión logística
lr_pipeline = Pipeline(stages=indexers + [assembler, scaler, lr])

print("Entrenando Regresión Logística...")

# Se entrena el pipeline completo con el DataFrame `supervised_df`.
lr_model = lr_pipeline.fit(supervised_df)
print("Modelo entrenado...")


--- Modelo 1: Regresión Logística ---
Entrenando Regresión Logística...


25/05/25 21:59:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/25 21:59:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
                                                                                

Modelo entrenado...


In [52]:
# Se Imprimen los valores de los coeficientes
lr_stage = lr_model.stages[-1]  # Última etapa es LogisticRegression
print("The coefficient of the model is :", lr_stage.coefficients)
print("The intercept of the model is :", lr_stage.intercept)

The coefficient of the model is : [-0.08414246503215402,-0.10305213628995627,-0.011979846792161571,-0.45523631995847075,-0.058556795214095445]
The intercept of the model is : -3.657629793952262


In [54]:
print("\n--- Evaluación de modelos ---")
# Preparar datos de prueba con la variable label
test_supervised = test_data.withColumn("label", 
                                    when(col("event_type") == "purchase", 1.0)
                                    .otherwise(0.0))

# Generar predicciones con el modelo de regresión logística previamente entrenado.
lr_predictions = lr_model.transform(test_supervised)

# Crear un evaluador para clasificación binaria.
# Se usará el área bajo la curva ROC (AUC) como métrica principal para evaluar qué tan bien distingue entre 0 y 1.
binary_evaluator = BinaryClassificationEvaluator(labelCol="label",rawPredictionCol="rawPrediction", metricName="areaUnderROC")

# Crear un evaluador adicional para calcular la precisión general (accuracy).
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="label", 
                                                      predictionCol="prediction", 
                                                      metricName="accuracy")

# Métricas para evaluar
lr_auc = binary_evaluator.evaluate(lr_predictions)
lr_accuracy = accuracy_evaluator.evaluate(lr_predictions)

print(f"\nResultados:")
print(f"Regresión Logística - AUC: {lr_auc:.4f}, Accuracy: {lr_accuracy:.4f}")


--- Evaluación de modelos ---


[Stage 195:====================================================>(144 + 2) / 146]


Resultados:
Regresión Logística - AUC: 0.5571, Accuracy: 0.9840


In [56]:
# Matriz de confusión
print("\nMatriz de confusión:")
confusion_matrix = lr_predictions.groupBy("label", "prediction").count()
confusion_matrix.show()


Matriz de confusión:


[Stage 197:====================================================>(145 + 1) / 146]

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       0.0|   35|
|  0.0|       0.0| 2159|
+-----+----------+-----+



#### 5.2 Aprendizaje No Supervisado

Para el problema de aprendizaje no supervisado haremos un análisis de clustering: agrupando productos por precio y popularidad

- Segmentación de prodcutos: Identificar productos populares y sus precios
- Objetivo: Marketing personalizado y estrategias diferenciadas
- Enfoque: Clustering basado en patrones de comportamiento


In [58]:
# Agrupa por `product_id` y calcula métricas agregadas:
# - Precio promedio del producto
# - Total de interacciones (vistas, carritos, compras)
# - Número de veces que el producto fue añadido al carrito
# - Número de veces que fue comprado
product_profiles = train_data.groupBy("product_id").agg(
    F.avg("price").alias("avg_price"),
    F.count("*").alias("total_views"),
    F.sum(when(col("event_type") == "cart", 1).otherwise(0)).alias("cart_count"),
    F.sum(when(col("event_type") == "purchase", 1).otherwise(0)).alias("purchase_count")
).filter(col("total_views") >= 10)  # Filtra productos con al menos 10 interacciones para reducir el ruido de clientes no frecuentes

# Agrega métricas derivadas de popularidad:
# - popularity_score: puntuación basada en interacción (views + 2*carritos + 5*compras)
# - conversion_rate: tasa de conversión (compras / vistas totales)
product_profiles = product_profiles.withColumn(
    "popularity_score", col("total_views") + col("cart_count") * 2 + col("purchase_count") * 5
).withColumn(
    "conversion_rate", col("purchase_count") / col("total_views")
)

# Cuántos productos se usarán para el clustering y una muestra de ellos
print(f"Productos para clustering: {product_profiles.count():,}")
print("Muestra de perfiles de productos:")
product_profiles.show(10)

# Selecciona las columnas que se usarán como características numéricas para el clustering
clustering_features = ["avg_price", "total_views", "popularity_score"]
print(f"Características seleccionadas: {clustering_features}")

Productos para clustering: 68
Muestra de perfiles de productos:


[Stage 206:====================================================>(145 + 1) / 146]

+----------+------------------+-----------+----------+--------------+----------------+--------------------+
|product_id|         avg_price|total_views|cart_count|purchase_count|popularity_score|     conversion_rate|
+----------+------------------+-----------+----------+--------------+----------------+--------------------+
|  22700129| 68.10166666666666|         12|         0|             0|              12|                 0.0|
|   1004659| 727.0933333333334|         21|         0|             1|              26|0.047619047619047616|
|   3700926| 67.98583333333333|         12|         2|             1|              21| 0.08333333333333333|
|   1005098|         139.95875|         16|         2|             0|              20|                 0.0|
|   1004781|263.97714285714284|         14|         1|             0|              16|                 0.0|
|   5100816|29.675238095238093|         21|         1|             2|              33| 0.09523809523809523|
|   1005105| 1366.5409302325

In [59]:
# Construye un pipeline de clustering en tres etapas:
# 1. VectorAssembler: combina las columnas numéricas en una sola columna de vectores
# 2. StandardScaler: normaliza los valores numéricos
# 3.KMeans: aplica el algoritmo de agrupamiento con 4 clusters
assembler = VectorAssembler(inputCols=clustering_features, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
kmeans = KMeans(featuresCol="scaledFeatures", k=4, seed=42)

# Construye el pipeline y lo ajusta (fit) al DataFrame `product_profiles`
pipeline = Pipeline(stages=[assembler, scaler, kmeans])
print("Entrenando modelo K-Means...")
model = pipeline.fit(product_profiles)

# Aplica el modelo de clustering a los perfiles de productos para asignar un clúster a cada producto
clustered_products = model.transform(product_profiles)
print("Modelo completado...")

Entrenando modelo K-Means...


[Stage 250:====================================================>(145 + 1) / 146]

Modelo completado...


In [60]:
print("\n--- Análisis de Clusters ---")

# Estadísticas por cluster
cluster_summary = clustered_products.groupBy("prediction").agg(
    F.count("*").alias("num_products"),
    F.avg("avg_price").alias("avg_price_cluster"),
    F.avg("total_views").alias("avg_popularity"),
    F.avg("conversion_rate").alias("avg_conversion")
).orderBy("prediction")

print("Resumen por cluster:")
cluster_summary.show()


--- Análisis de Clusters ---
Resumen por cluster:


[Stage 256:====================================================>(145 + 1) / 146]

+----------+------------+------------------+------------------+--------------------+
|prediction|num_products| avg_price_cluster|    avg_popularity|      avg_conversion|
+----------+------------+------------------+------------------+--------------------+
|         0|          16| 864.8541590319077|           17.0625| 0.01933768750629216|
|         1|           4|370.78704276139734|             73.25| 0.03753284192088746|
|         2|          41|218.95489361534854|14.292682926829269| 0.02686344143661217|
|         3|           7| 354.5078094808635|              38.0|0.022876542737169915|
+----------+------------+------------------+------------------+--------------------+



In [61]:
# Se recopilan los resultados agregados de los clústeres desde un DataFrame Spark a una lista local de Python.
cluster_data = cluster_summary.collect()

# Se itera sobre cada fila del resumen de clústeres para analizar y clasificar los grupos
for row in cluster_data:
    cluster_id = int(row['prediction'])
    count = int(row['num_products'])
    price = float(row['avg_price_cluster'])
    popularity = float(row['avg_popularity'])
    conversion = float(row['avg_conversion'])
    
    # Clasificación simple
    if price > 500:
        cluster_type = "🏆 Premium"
        description = "Productos caros, baja popularidad, alta conversión"
    elif popularity > 100:
        cluster_type = "🔥 Populares"
        description = "Mucha actividad, precios medios"
    elif conversion > 0.05:
        cluster_type = "💰 Best Seller"
        description = "Buena conversión, productos exitosos"
    else:
        cluster_type = "📦 Basics"
        description = "Actividad normal, precios accesibles"
    
    print(f"\nCluster {cluster_id}: {cluster_type}")
    print(f"  • Productos: {count:,}")
    print(f"  • Precio promedio: ${price:.2f}")
    print(f"  • Popularidad: {popularity:.1f} views")
    print(f"  • Conversión: {conversion:.3f} ({conversion*100:.1f}%)")
    print(f"  • {description}")

[Stage 262:====================================================>(145 + 1) / 146]


Cluster 0: 🏆 Premium
  • Productos: 16
  • Precio promedio: $864.85
  • Popularidad: 17.1 views
  • Conversión: 0.019 (1.9%)
  • Productos caros, baja popularidad, alta conversión

Cluster 1: 📦 Basics
  • Productos: 4
  • Precio promedio: $370.79
  • Popularidad: 73.2 views
  • Conversión: 0.038 (3.8%)
  • Actividad normal, precios accesibles

Cluster 2: 📦 Basics
  • Productos: 41
  • Precio promedio: $218.95
  • Popularidad: 14.3 views
  • Conversión: 0.027 (2.7%)
  • Actividad normal, precios accesibles

Cluster 3: 📦 Basics
  • Productos: 7
  • Precio promedio: $354.51
  • Popularidad: 38.0 views
  • Conversión: 0.023 (2.3%)
  • Actividad normal, precios accesibles


In [65]:
# Guardar los modelos
base_path = "/Users/pauescalante/Documents/Maestria/Trimestre 7/BigData/big-data-act/Models"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

lr_model_path = f"{base_path}/logistic_regression_model_{timestamp}"
try:
    lr_model.write().overwrite().save(lr_model_path)
    print(f"Modelo LR guardado en: {lr_model_path}")
except Exception as e:
    print(f"Error guardando LR: {e}")

# Guardar modelo K-Means completo
kmeans_model_path = f"{base_path}/kmeans_model_{timestamp}"
try:
    # Asumiendo que 'model' es tu pipeline de clustering
    model.write().overwrite().save(kmeans_model_path)
    print(f"Modelo K-Means guardado en: {kmeans_model_path}")
except Exception as e:
    print(f"Error guardando K-Means: {e}")

Modelo LR guardado en: /Users/pauescalante/Documents/Maestria/Trimestre 7/BigData/big-data-act/Models/logistic_regression_model_20250525_223454
Modelo K-Means guardado en: /Users/pauescalante/Documents/Maestria/Trimestre 7/BigData/big-data-act/Models/kmeans_model_20250525_223454


### **6. Conclusión**

En esta actividad se exploraron técnicas de aprendizaje supervisado y no supervisado utilizando PySpark sobre un conjunto de datos reales de comportamiento en e-commerce. En el caso supervisado, se entrenó un modelo de regresión logística para predecir la probabilidad de compra, enfrentando el reto de una distribución altamente desbalanceada. Para el aprendizaje no supervisado, se aplicó K-Means para segmentar productos en clústeres basados en métricas de precio, popularidad y conversión. Esta segmentación permite identificar oportunidades estratégicas para acciones de marketing y posicionamiento de productos. El uso de pipelines y transformaciones escalables con PySpark permitió construir un flujo de trabajo eficiente y reproducible para el análisis de grandes volúmenes de datos.